import libraries

In [1]:
import tmdbsimple as tmdb
import requests
import pandas as pd
import numpy as np
import time
from ast import literal_eval

#import libraries for vectorizing the plots
from sklearn.feature_extraction.text import CountVectorizer
import re
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
#Grid search lib for modelling and SVM
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer
from sklearn.metrics import classification_report


In [3]:
movie_df = pd.read_csv('data/movie_df_with_imdb.csv',encoding="utf-8")

In [4]:
movie_df.head(2)

,tmdb_id,imdb_id,tmdb_genres,imdb_genres,binary_tmdb,binary_imdb,tmdb_plot,imdb_plot,popularity,release_date,title,vote_average,vote_count
0,278,tt0111161,"[18, 80]","[80, 18]",[0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0],[0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0],Framed in the 1940s for the double murder of h...,Chronicles the experiences of a formerly succe...,28.527767,1994-09-23,The Shawshank Redemption,8.5,9773
1,238,tt0068646,"[18, 80]","[80, 18]",[0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0],[0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0],"Spanning the years 1945 to 1955, a chronicle o...",When the aging head of a famous crime family d...,36.965452,1972-03-14,The Godfather,8.5,7394


In [5]:
movie_df.loc[0,:]
#movie1['tmdb_id']

tmdb_id                                                       278
imdb_id                                                 tt0111161
tmdb_genres                                              [18, 80]
imdb_genres                                              [80, 18]
binary_tmdb                 [0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
binary_imdb                 [0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
tmdb_plot       Framed in the 1940s for the double murder of h...
imdb_plot       Chronicles the experiences of a formerly succe...
popularity                                                28.5278
release_date                                           1994-09-23
title                                    The Shawshank Redemption
vote_average                                                  8.5
vote_count                                                   9773
Name: 0, dtype: object

In [6]:
complete_plot =[]
for i in range(len(movie_df)):
    movie1=movie_df.loc[i,:]
    id=movie1['tmdb_id']
    tmdb_plot=movie1['tmdb_plot']
    tmdb_plot=tmdb_plot.replace(',','')
    tmdb_plot=tmdb_plot.replace('.','')
    complete_plot.append(tmdb_plot)

In [7]:
print ("Print the 1st tmdb plot ")
print (complete_plot[0])
print ("============================")
print ("The length of  the 1st tmdb plot ")
print (len(complete_plot))

Print the 1st tmdb plot 
Framed in the 1940s for the double murder of his wife and her lover upstanding banker Andy Dufresne begins a new life at the Shawshank prison where he puts his accounting skills to work for an amoral warden During his long stretch in prison Dufresne comes to be admired by the other inmates -- including an older prisoner named Red -- for his integrity and unquenchable sense of hope
The length of  the 1st tmdb plot 
1000


In [19]:
#Tf-IDF is being used to reduce dimensionality of the input matrix X 
# "Bag of words" representation
vectorize=CountVectorizer(max_df=0.95, min_df=0.05)
# min_df  exclude words that occur rarely
# max_df  exclude words that occur too frequently
X=vectorize.fit_transform(complete_plot)

TFID_transform = TfidfTransformer()
X_TFID_transformed = TFID_transform.fit_transform(X)
X_TFID_transformed.shape

(1000, 83)

In [20]:
list_genres=[]

for i in range(len(movie_df)):
    movie=movie_df.loc[i,:]
    id=movie['tmdb_id']
    genreIds=movie['tmdb_genres']
    list_genres.append(genreIds)
   

In [21]:
list_genres[10]

'[18, 80]'

In [22]:
mlb=MultiLabelBinarizer()
y=mlb.fit_transform(list_genres)
print (y[1])       #last element
#print (mlb.classes_)

[1 1 1 1 0 0 0 0 0 0 1 0 1 1]


In [24]:
#Split X and Y for train and test dataset with 70:30 ratio
mask = np.random.rand(X_TFID_transformed.shape[0]) < 0.7

In [25]:
X_train =X_TFID_transformed[mask]
X_test =X_TFID_transformed[~mask]
y_train =y[mask]
y_test =y[~mask]

In [27]:
param_grid = [
  {'C':[0.01, 0.1, 1.0], 'kernel': ['linear']},
  {'C':[0.01, 0.1, 1.0], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
 ]
gridCV = GridSearchCV(SVC(class_weight='balanced'), param_grid, scoring=make_scorer(f1_score, average='micro'))
grid_fit = OneVsRestClassifier(gridCV)

grid_fit.fit(X_train, y_train)

C:\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label 12 is present in all training examples.
  str(classes[c]))
C:\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label 13 is present in all training examples.
  str(classes[c]))


OneVsRestClassifier(estimator=GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)...fit=True, return_train_score='warn',
       scoring=make_scorer(f1_score, average=micro), verbose=0),
          n_jobs=1)

In [28]:
key = '38d770fa1c4e05950146f1ad5caa789f'
payload={}
url = "https://api.themoviedb.org/3/genre/movie/list?api_key={0}&language=en-US".format(key)
response = requests.request("GET", url, data=payload).json()

id_to_genre = dict(zip([i['id'] for i in response['genres']],
                     [i['name'] for i in response['genres']]))

genre_to_id = dict(zip([i['name'] for i in response['genres']],
                       [i['id'] for i in response['genres']]))

id_to_genre

{12: 'Adventure',
 14: 'Fantasy',
 16: 'Animation',
 18: 'Drama',
 27: 'Horror',
 28: 'Action',
 35: 'Comedy',
 36: 'History',
 37: 'Western',
 53: 'Thriller',
 80: 'Crime',
 99: 'Documentary',
 878: 'Science Fiction',
 9648: 'Mystery',
 10402: 'Music',
 10749: 'Romance',
 10751: 'Family',
 10752: 'War',
 10770: 'TV Movie'}

In [29]:
target_genres=[u'Action',
u'Adventure',
u'Animation',
u'Comedy',
u'Crime',
u'Documentary',
u'Drama',
u'Family',
u'Fantasy',
u'History',
u'Horror',
u'Music',
u'Mystery',
u'Romance',
u'Science Fiction',
u'TV Movie',
u'Thriller',
u'War',
u'Western']

In [30]:
y_pred =grid_fit.predict(X_test)

print (classification_report(y_test, y_pred, target_names=target_genres))

                 precision    recall  f1-score   support

         Action       0.81      0.65      0.72       243
      Adventure       0.81      0.65      0.72       243
      Animation       0.54      0.56      0.55       141
         Comedy       0.00      0.00      0.00       257
          Crime       0.51      0.59      0.55       115
    Documentary       0.59      0.54      0.56       167
          Drama       0.32      1.00      0.48        95
         Family       0.63      0.68      0.65       176
        Fantasy       0.26      0.51      0.34        70
        History       0.56      0.55      0.56       137
         Horror       0.86      0.70      0.77       252
          Music       0.24      1.00      0.38        71
        Mystery       1.00      1.00      1.00       300
        Romance       1.00      1.00      1.00       300

    avg / total       0.66      0.67      0.65      2567



C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1428: UserWarning: labels size, 14, does not match size of target_names, 19
  .format(len(labels), len(target_names))
C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
